# Raw Dataset

## Details about Notebook

This Notebook will import the Raw Dataset properly and prepare it for training a model. 

### Importing Libraries

In [ ]:
import os
import math
import json
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (
    Conv2D,
    MaxPool2D,
    Flatten,
    Dense,
    Dropout,
    GlobalAveragePooling2D,
    BatchNormalization,
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import image_dataset_from_directory, load_img, img_to_array
from tensorflow.keras.applications import InceptionV3
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    ConfusionMatrixDisplay
)

In [ ]:
# Global Variables

NUM_CLASSES = 38
IMAGE_SIZE = 224
BATCH_SIZE = 32

### Importing Dataset

#### Dataset Preparation

In [ ]:
# Paths to the dataset folders
TRAIN_DIR = "../datasets/cropped_plant_village_dataset/train"
VALID_DIR = "../datasets/cropped_plant_village_dataset/valid"
SAMPLE_IMAGE = "../datasets/cropped_plant_village_dataset/sample_image.JPG"

In [ ]:
# Global Variables


##### Training Set

In [ ]:
training_set = image_dataset_from_directory(
    TRAIN_DIR,
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=BATCH_SIZE,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
)

##### Validation Set

In [ ]:
validation_set = tf.keras.utils.image_dataset_from_directory(
    VALID_DIR,
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=BATCH_SIZE,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
)

In [ ]:
test_set = tf.keras.utils.image_dataset_from_directory(
    VALID_DIR,
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=1,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    shuffle=False,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
)

#### Dataset Details

In [ ]:
# Function to count images in each class
def count_images_in_classes(dataset_dir):
    """
    Counts the number of images in each class within a dataset directory.

    Args:
        dataset_dir (str): The path to the dataset directory.

    Returns:
        dict: A dictionary where the keys are the class names and the values are the number of images in each class.
    """
    class_counts = {}
    for class_name in os.listdir(dataset_dir):
        class_path = os.path.join(dataset_dir, class_name)
        if os.path.isdir(class_path):
            class_counts[class_name] = len(os.listdir(class_path))
    return class_counts


# Count images in training and validation sets
train_class_counts = count_images_in_classes(TRAIN_DIR)
valid_class_counts = count_images_in_classes(VALID_DIR)

# Create a DataFrame for better visualization
df = pd.DataFrame(
    {
        "Class": list(train_class_counts.keys()),
        "Training Images": list(train_class_counts.values()),
        "Validation Images": 
            list(valid_class_counts.values())
        # [
            # valid_class_counts.get(cls, 0) for cls in train_class_counts.keys()
        # ],
    }
).sort_values(by="Class", ascending=True)

In [ ]:
# Display the DataFrame
print(df)

In [ ]:
# Plot the class distribution with adjustments for readability
df.plot(
    x="Class", kind="bar", stacked=True, figsize=(20, 8), title="Class Distribution"
)
plt.ylabel("Number of Images")
plt.xlabel("Class")

# Rotate x-ticks for better readability
plt.xticks(rotation=90, ha="center")

# Adjust layout to prevent clipping of labels
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
# Function to visualize one image per class
def visualize_sample_images(dataset_dir):
    """
    Visualizes one sample image per class in the dataset directory.

    Args:
        dataset_dir (str): The path to the dataset directory.

    Displays a grid of images, with one image per class, using matplotlib.
    """
    class_names = os.listdir(dataset_dir)
    class_names.sort()  # Sort for consistent order

    # Calculate the number of rows and columns for the subplot grid
    num_classes = len(class_names)
    num_cols = 5  # You can adjust this number
    num_rows = math.ceil(num_classes / num_cols)

    plt.figure(figsize=(num_cols * 3, num_rows * 3))
    for i, class_name in enumerate(class_names, start=1):
        class_path = os.path.join(dataset_dir, class_name)
        image_path = os.path.join(
            class_path, os.listdir(class_path)[0]
        )  # Get the first image in the class
        img = plt.imread(image_path)

        plt.subplot(num_rows, num_cols, i)  # Adjust grid size dynamically
        plt.imshow(img)
        plt.title(class_name)
        plt.axis("off")

    plt.tight_layout()
    plt.show()

In [ ]:
# Visualize sample images from training set
visualize_sample_images(TRAIN_DIR)

### Image Preprocessing

#### Normalization

In [ ]:
normalize = tf.keras.layers.Rescaling(1.0 / 255)

# Apply normalization on both Training and Validation set
normalized_training_set = training_set.map(lambda x, y: (normalize(x), y))
normalized_validation_set = validation_set.map(lambda x, y: (normalize(x), y))
normalized_test_set = test_set.map(lambda x, y: (normalize(x), y))

#### Image Augmentation

In [ ]:
# Apply augmentation to the training set
augmented_training_set = normalized_training_set

#### Image Enhancement (not implemented)

In [ ]:
# Image Enhancements have not strongly proven to increase the accuracy

## Conclusion